In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google import colab
%cd /content/drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [118]:
import pandas as pd

In [119]:
articles_df = pd.read_parquet('students_articles_enriched_1.parquet.gzip', engine='pyarrow')
pageviews_df = pd.read_parquet('students_pageviews_2021-01-01.parquet.gzip', engine='pyarrow')
# page_views_count_df = pageviews_df['article_drive_id'].value_counts().rename_axis('article_drive_id').reset_index(name='total_count')
# page_views_by_time_spent_df = pageviews_df.groupby('article_drive_id').time_engaged_in_s.apply(lambda g: g.sum()).reset_index(name ='total_time_spent')
# dd = pd.merge(page_views_count_df,page_views_by_time_spent_df,'right','article_drive_id')
agdf = (pageviews_df
        .groupby(['article_drive_id'])
        .agg(total_time_engaged_in_s=('time_engaged_in_s', 'sum'),total_count=('article_drive_id', 'count'))
        .reset_index(drop=False)
)
agdf = agdf.assign(time_engaged_in_m = lambda x: (x['total_time_engaged_in_s']/60))

In [120]:
import plotly.express as px
fig = px.scatter(agdf, x=agdf["total_time_engaged_in_s"], y=agdf["total_count"], hover_name=agdf["article_drive_id"], log_x=True)
fig.update_layout(title_text='total time spent per article')
fig.show()

In [121]:
sub_articles_df = articles_df[['article_drive_id', 'article_full_text', 'topic']].copy()
sub_articles_df['article_length'] = sub_articles_df.article_full_text.str.len()
newPd = pd.merge(sub_articles_df,agdf,'left','article_drive_id')
resultdf = (newPd
        .groupby(['topic'])
        .agg(total_time_engaged_in_s=('total_time_engaged_in_s', 'sum'),total_count=('article_drive_id', 'count'))
        .reset_index(drop=False)
)
resultdf = resultdf[resultdf.topic != 0]
resultdf = resultdf.assign(time_engaged_in_minutes = lambda x: (x['total_time_engaged_in_s']/60))

In [122]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    x=resultdf['topic'],
    y=resultdf['total_count'],
    name='topic_count',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=resultdf['topic'],
    y=resultdf['time_engaged_in_minutes'],
    name='time_engaged_in_minutes',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45, title_text='time/article per topic')
fig.show()

In [123]:
newPd1 = pd.merge(sub_articles_df,agdf,'right','article_drive_id')
resultdf1 = (newPd1
        .groupby(['topic', 'article_length'])
        .agg(total_time_engaged_in_s=('total_time_engaged_in_s', 'sum'))
        .reset_index(drop=False)
)
resultdf1 = resultdf1[resultdf1.topic != 0]

In [124]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    x=resultdf1['topic'],
    y=resultdf1['article_length'],
    name='article_length',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=resultdf['topic'],
    y=resultdf['total_time_engaged_in_s'],
    name='total_time_engaged_in_s',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45, title_text='time/article-length per topic')
fig.show()